<a href="https://colab.research.google.com/github/liavnadam/Deep-Dream/blob/main/SAM2_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
HOME = '/content/drive/MyDrive'

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/segment-anything-2
#installing segment-anything-2
!pip install -e .

!mkdir -p {HOME}/checkpoints

/content/drive/MyDrive/segment-anything-2
Obtaining file:///content/drive/MyDrive/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.3 MB/s eta 0:00:00
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=SAM_2-1.0-0.editable-cp310-cp310-linux_x86_64.whl size=12331 sha256=dea4eceeaa7d97463ad4aa42b8df2447d18f769be6dfd2cafe02be65474a91d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-n8gbtpzd/whe

In [ ]:
!pip install Flask-Cors
!pip install pyngrok

import sys
import cv2
import torch
import numpy as np
import base64
import random
from io import BytesIO
from PIL import Image
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading
import traceback
import matplotlib.pyplot as plt
import subprocess
import io
import random

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

In [ ]:
torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()

if torch.cuda.get_device_properties(0).major >= 8:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINT = f"{HOME}/checkpoints/sam2_hiera_large+.pt"
CONFIG = "sam2_hiera_l.yaml"
sam2_model = build_sam2(CONFIG, CHECKPOINT, device=DEVICE, apply_postprocessing=False)
predictor = SAM2ImagePredictor(sam2_model)

In [ ]:
def show_mask(mask, ax, random_color=False, borders=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2)
    ax.imshow(mask_image)

def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()

In [ ]:
# Set up ngrok
def start_ngrok():
    url = ngrok.connect(8000)
    print("ngrok tunnel URL is:", url)
    print(f"ngrok tunnel available at: {url}")

# Functions for image processing and segmentation
# Constants for resizing
MAX_WIDTH = 1024
MAX_HEIGHT = 768

def adjust_points(points, resize_ratio):
    adjusted_points = []
    for point in points:
        x = int(point['x'] * resize_ratio[0])
        y = int(point['y'] * resize_ratio[1])
        adjusted_points.append({'x': x, 'y': y, 'label': point.get('label', 0)})
    return adjusted_points


def resize_image(image, max_width, max_height):
    h, w = image.shape[:2]
    aspect_ratio = w / h
    if w > max_width:
        w = max_width
        h = int(w / aspect_ratio)
    if h > max_height:
        h = max_height
        w = int(h * aspect_ratio)
    return cv2.resize(image, (w, h))


def post_process_segmentation(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

def get_image_from_base64(base64_str):
    try:
        encoded_data = base64_str.split(',')[1]
        nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        if img is None:
            raise ValueError("Image decode failed")
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    except Exception as e:
        print(f"Error decoding image: {e}")
        return None

def get_base64_encoded_image(image_path):
    try:
        with open(image_path, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding image: {e}")
        return None


def annotate_image(image, masks, mode='single', opacity=0.6):
    annotated_image = image.copy()

    if mode == 'multiple':
        # Generate a list of random colors, one for each mask
        colors = [
            [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
            for _ in range(len(masks))
        ]
        for mask, color in zip(masks, colors):
            mask = mask.astype(bool)  # Ensure the mask is boolean
            # Blend the color with the original image using the specified opacity
            annotated_image[mask] = (
                opacity * np.array(color, dtype=np.uint8) + (1 - opacity) * annotated_image[mask]
            ).astype(np.uint8)
    else:
        # Single object mode: Use green color
        color = np.array([0, 255, 0], dtype=np.uint8)  # Green color for the mask
        for mask in masks:
            mask = mask.astype(bool)  # Ensure the mask is boolean
            # Blend the green color with the original image using the specified opacity
            annotated_image[mask] = (
                opacity * color + (1 - opacity) * annotated_image[mask]
            ).astype(np.uint8)

    return annotated_image

In [ ]:
app = Flask(__name__)
CORS(app, supports_credentials=True)

In [ ]:
@app.route('/segment', methods=['POST'])
def segment_image():
    try:
        data = request.get_json()
        image_base64 = data['imageData']
        points = data['points']
        mode = data.get('mode', 'single')  # Default to single object

        if not points or not isinstance(points, list):
            return jsonify({'error': 'Invalid points data'}), 400

        image_array = get_image_from_base64(image_base64)
        if image_array is None:
            return jsonify({'error': 'Invalid image data'}), 400

        # Resize image and get resize ratio
        original_height, original_width = image_array.shape[:2]
        resized_image = resize_image(image_array, MAX_WIDTH, MAX_HEIGHT)
        resized_height, resized_width = resized_image.shape[:2]
        resize_ratio = (resized_width / original_width, resized_height / original_height)

        temp_images_dir = os.path.join(os.getcwd(), 'temp_images')
        if not os.path.exists(temp_images_dir):
            os.makedirs(temp_images_dir)
        image_path = os.path.join(temp_images_dir, 'temp_image.jpg')

        cv2.imwrite(image_path, cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR))

        # Adjust points coordinates
        adjusted_points = adjust_points(points, resize_ratio)

        valid_points = []
        labels = []
        for point in adjusted_points:
            try:
                x, y, label = int(point['x']), int(point['y']), int(point['label'])
                if 0 <= x < resized_image.shape[1] and 0 <= y < resized_image.shape[0]:
                    valid_points.append((x, y))
                    labels.append(label)
            except KeyError as e:
                print(f"Error accessing point: {point}, error: {e}")

        input_point = np.array(valid_points)
        input_label = np.array(labels)

        if mode == 'multiple':
            # Convert the points and labels to the required format
            pts_batch = [np.array(input_point).reshape(-1, 1, 2).astype(np.float32)]
            labels_batch = [np.ones((len(input_point), 1), dtype=np.int32)]  # Assuming label 1 for all points

            img_batch = [resized_image]
            predictor.set_image_batch(img_batch)

            # Predict masks and scores
            masks_batch, scores_batch, _ = predictor.predict_batch(
                pts_batch, labels_batch,
                box_batch=None,
                multimask_output=True
            )

            # Select the best single mask per object
            best_masks = []
            for masks, scores in zip(masks_batch, scores_batch):
                best_masks.append(masks[np.arange(len(masks)), np.argmax(scores, axis=-1)])

            masks = [mask for sublist in best_masks for mask in sublist]

        else:
            predictor.set_image(resized_image)
            # Single object segmentation
            masks, scores, logits = predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                multimask_output=True,
            )
            mask_input = logits[np.argmax(scores), :, :]
            masks, scores, _ = predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                mask_input=mask_input[None, :, :],
                multimask_output=False,
            )

        if len(masks) > 0:
            # Pass mode parameter to annotate_image
            mask_annotated_image = annotate_image(resized_image, masks, mode)
            mask_image_path = os.path.join(temp_images_dir, '1.jpeg')
            cv2.imwrite(mask_image_path, cv2.cvtColor(mask_annotated_image, cv2.COLOR_RGB2BGR))
            mask_image = post_process_segmentation(mask_annotated_image)
            result_base64 = get_base64_encoded_image(mask_image_path)

            return jsonify({'status': 'success', 'segmentedImage': result_base64})
        else:
            return jsonify({'status': 'success', 'segmentedImage': None})

    except Exception as e:
        print(f"Error during processing: {e}")
        traceback.print_exc()
        return jsonify({'error': 'Server error', 'message': str(e)}), 500



if __name__ == '__main__':
    ngrok.set_auth_token("2l6r6vCbENywTecJF2Hwoe8Wqxh_6DC3QZbyZatFAMwowGx5i")
    public_url = ngrok.connect(8000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:8000\"")
    app.run(host='0.0.0.0', port=8000)


 * ngrok tunnel "NgrokTunnel: "https://5840-34-143-164-181.ngrok-free.app" -> "http://localhost:8000"" -> "http://127.0.0.1:8000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Aug/2024 20:55:27] "OPTIONS /segment HTTP/1.1" 200 -
/content/drive/MyDrive/segment-anything-2/sam2/modeling/sam/transformer.py:270: UserWarning: Memory efficient kernel not used because: (Triggered internally at ../aten/src/ATen/native/transformers/cuda/sdp_utils.cpp:718.)
  out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_p)
/content/drive/MyDrive/segment-anything-2/sam2/modeling/sam/transformer.py:270: UserWarning: Memory Efficient attention has been runtime disabled. (Triggered internally at ../aten/src/ATen/native/transformers/sdp_utils_cpp.h:495.)
  out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_p)
/content/drive/MyDrive/segment-anything-2/sa